In [50]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib.cm as cm
import math
import sympy as sm
%matplotlib qt

In [98]:
#System definition
def xdot(y, t, a, b, c):
    x1, x2, x3 = y
    return [ - x2 - x3,
               x1 + a * x2,
               b + x3 * (x1 - c)]

def get_period(sol):
    """
    Gets the Minimum period of the solution
    """
    x0, y0, z0 = sol.y[:, 0]
    init = np.array([x0, y0, z0])
    x, y, z = sol.y
    dist = np.zeros(x.shape[0])
    for i in range(len(dist)):
        ipoint = np.array([x[i], y[i], z[i]])
        dist[i] = math.dist(init, ipoint)

    dist_arr = []
    for i in range(len(dist[2:])):
        if dist[i-1] > dist[i] and dist[i+1] > dist[i] and dist[i] != 0:
            dist_arr.append((sol.t[i], dist[i]))
    
    sorted_tuples = sorted(dist_arr, key=lambda x: (x[1], x[0]))
    
    return sorted_tuples[0][0]

In [208]:
#define time range
timespan = (0., 100.)
init = 1 #number of initial conditions

#Paramters
a = 0.2
b = 0.2

#initial conditions
x0 = 4.7
y0 = -1.1430
z0 = 0.85
i = 0

colors = np.linspace(0, 1 / 1, 1)
fig = plt.figure()
axs = fig.add_subplot(111, projection='3d')

initial_conditions = [x0, y0, z0]

c = 5.3
sol = solve_ivp(lambda t, y, a, b, c: xdot(y, t, a, b, c), t_span=timespan, y0=initial_conditions, max_step=0.01, args=(a, b, c))
#Picking coords of solution
x, y, z = sol.y
axs.plot(x, y, z, color = cm.hot(min(colors[i]**0.4, 0.7)), label = x0)
axs.view_init(elev=20, azim=45) 
axs.set_xlabel('X-axis')
axs.set_ylabel('Y-axis')
axs.set_zlabel('Z-axis')

print(f'Period: {get_period(sol)}')
# Show the plot
plt.show()


Period: 17.409999999999922


In [204]:
c = sm.symbols('c')

mon_mat = sm.Matrix([[0, -1, -1], 
                    [1,  a,  0] ,
                    [z0, 0,x0 - c]])

pts = []
for k in np.linspace(2, 7, 10):
    new_mat = mon_mat.subs(c, k)
    eigenvals = new_mat.eigenvals()


    i = 0
    for eig in eigenvals.keys():
        real, imag = eig.as_real_imag()
        pts.append((real, imag, k))

In [207]:
i = 0
for pt in pts:    
    plt.scatter(pt[0], pt[1], label = pt[2], color = cm.hot((pt[2]+3.6*pt[2])/(pt[2]**3)))

plt.legend()
plt.title("Eigenvalues for different c")
plt.xlabel("Real")
plt.ylabel("Imaginary")
plt.show()

In [85]:
sol.sol